In [1]:
#=========================================================================================================
#================================ 0. MODULE


# Class
from numerai import Numerai

# Architecture
from architecture import models, nCores


#=========================================================================================================
#================================ 1. CLASS


stacking = Numerai(week=110)


#=========================================================================================================
#================================ 2. FEATURE ENGINEERING


# metafeature = ['variance', 'mean', 'distance']
# stages = [2, 0, 2]
# stacking.add_metafeature(metafeature, stages)


#=========================================================================================================
#================================ 3. TRAINING MODEL


###### MACHINE LEARNING
stacking.add_model(models)
stacking.fit_tune(nCores, stageNumber=1, neuralNetworkCompiler=False, evaluate=True, interaction=None)


---------------------------------------------
>> Loading data...done

Xtrain1: (157445, 50) 
Ytrain1: (157445,) 
Xtrain2: (158232, 50) 
Ytrain2: (158232,) 
Xtest: (77936, 50) 
Ytest: (77936,) 
Xvalid: (46362, 50) 
Yvalid: (46362,) 
Submit data: (243283, 50)


---------------------------------------------
>> Processing first stage

>> Processing ExtraTrees1 ------

Step 1...done in 0h 1m 0s
log loss : 0.6930542843089739

Step 2...done in 0h 0m 58s
log loss : 0.6929652177180379

Step 3...done in 0h 1m 0s
log loss : 0.6929716218430513

Step 4...done in 0h 0m 50s
log loss : 0.6929227555221473

Step 5...done in 0h 0m 49s
log loss : 0.6930320371706428

Step 6...done in 0h 0m 52s
log loss : 0.6930033903971393

Step 7...done in 0h 0m 53s
log loss : 0.693059991653412

Step 8...done in 0h 0m 51s
log loss : 0.6930647889119426

Step 9...done in 0h 0m 56s
log loss : 0.6930241070740396

Step 10...done in 0h 0m 58s
log loss : 0.6930305708571719


>> Processing ExtraTrees2 ------

Step 1...done in 0h

In [ ]:
#=========================================================================================================
#================================ 4. PREDICTION

# stacking.fit_tune(nCores, neuralNetworkCompiler=False, evaluate=False, interaction=None)
# stacking.neuralNetwork(learningRate=0.0001, batch=64, epoch=2, cvNumber=2, displayStep=10000, evaluate=False, useGPU=True)
# stacking.submit(5, submissionNumber=2, week=109)